In [7]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

IMG_SIZE = 512

def load_images_from_folder(folder_path, label, img_size):
    data = []
    labels = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = load_img(img_path, target_size=(img_size, img_size))
        img_array = img_to_array(img) / 255.0
        data.append(img_array)
        labels.append(label)
    return data, labels

# Load training data
train_is_path = "ice_cream/train/ice_cream"
train_not_path = "ice_cream/train/non_ice_cream"
train_data_is, labels_is = load_images_from_folder(train_is_path, 1, IMG_SIZE)
train_data_not, labels_not = load_images_from_folder(train_not_path, 0, IMG_SIZE)

X_train = np.array(train_data_is + train_data_not)
y_train = np.array(labels_is + labels_not)

# Load testing data
test_is_path = "ice_cream/test/ice_cream"
test_not_path = "ice_cream/test/non_ice_cream"
test_data_is, test_labels_is = load_images_from_folder(test_is_path, 1, IMG_SIZE)
test_data_not, test_labels_not = load_images_from_folder(test_not_path, 0, IMG_SIZE)

X_test = np.array(test_data_is + test_data_not)
y_test = np.array(test_labels_is + test_labels_not)


KeyboardInterrupt: 

In [ ]:
# Rapport generator funktioner
from time import time
import numpy as np
import sys

from sklearn import svm
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import datasets

# from libitmal import dataloaders as itmaldataloaders # Needed for load of iris, moon and mnist

currmode="N/A" # GLOBAL var!

def SearchReport(model): 
    
    def GetBestModelCTOR(model, best_params):
        def GetParams(best_params):
            ret_str=""          
            for key in sorted(best_params):
                value = best_params[key]
                temp_str = "'" if str(type(value))=="<class 'str'>" else ""
                if len(ret_str)>0:
                    ret_str += ','
                ret_str += f'{key}={temp_str}{value}{temp_str}'  
            return ret_str          
        try:
            param_str = GetParams(best_params)
            return type(model).__name__ + '(' + param_str + ')' 
        except:
            return "N/A(1)"
        
    print("\nBest model set found on train set:")
    print()
    print(f"\tbest parameters={model.best_params_}")
    print(f"\tbest '{model.scoring}' score={model.best_score_}")
    print(f"\tbest index={model.best_index_}")
    print()
    print(f"Best estimator CTOR:")
    print(f"\t{model.best_estimator_}")
    print()
    try:
        print(f"Grid scores ('{model.scoring}') on development set:")
        means = model.cv_results_['mean_test_score']
        stds  = model.cv_results_['std_test_score']
        i=0
        for mean, std, params in zip(means, stds, model.cv_results_['params']):
            print("\t[%2d]: %0.3f (+/-%0.03f) for %r" % (i, mean, std * 2, params))
            i += 1
    except:
        print("WARNING: the random search do not provide means/stds")
    
    global currmode                
    assert "f1_micro"==str(model.scoring), f"come on, we need to fix the scoring to be able to compare model-fits! Your scoreing={str(model.scoring)}...remember to add scoring='f1_micro' to the search"   
    return f"best: dat={currmode}, score={model.best_score_:0.5f}, model={GetBestModelCTOR(model.estimator,model.best_params_)}", model.best_estimator_ 

def ClassificationReport(model, X_test, y_test, target_names=None):
    assert X_test.shape[0]==y_test.shape[0]
    print("\nDetailed classification report:")
    print("\tThe model is trained on the full development set.")
    print("\tThe scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)                 
    print(classification_report(y_true, y_pred, target_names=target_names))
    print()
    
def FullReport(model, X_test, y_test, t):
    print(f"SEARCH TIME: {t:0.2f} sec")
    beststr, bestmodel = SearchReport(model)
    ClassificationReport(model, X_test, y_test)    
    print(f"CTOR for best model: {bestmodel}\n")
    print(f"{beststr}\n")
    return beststr, bestmodel
    
# def LoadAndSetupData(mode, test_size=0.3):
#     assert test_size>=0.0 and test_size<=1.0
    
#     def ShapeToString(Z):
#         n = Z.ndim
#         s = "("
#         for i in range(n):
#             s += f"{Z.shape[i]:5d}"
#             if i+1!=n:
#                 s += ";"
#         return s+")"

#     global currmode
#     currmode=mode
#     print(f"DATA: {currmode}..")
    
#     if mode=='moon':
#         X, y = itmaldataloaders.MOON_GetDataSet(n_samples=5000, noise=0.2)
#         itmaldataloaders.MOON_Plot(X, y)
#     elif mode=='mnist':
#         X, y = itmaldataloaders.MNIST_GetDataSet(load_mode=0)
#         if X.ndim==3:
#             X=np.reshape(X, (X.shape[0], -1))
#     elif mode=='iris':
#         X, y = itmaldataloaders.IRIS_GetDataSet()
#     else:
#         raise ValueError(f"could not load data for that particular mode='{mode}', only 'moon'/'mnist'/'iris' supported")
        
#     print(f'  org. data:  X.shape      ={ShapeToString(X)}, y.shape      ={ShapeToString(y)}')

#     assert X.ndim==2
#     assert X.shape[0]==y.shape[0]
#     assert y.ndim==1 or (y.ndim==2 and y.shape[1]==0)    
    
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=0, shuffle=True
#     )
    
#     print(f'  train data: X_train.shape={ShapeToString(X_train)}, y_train.shape={ShapeToString(y_train)}')
#     print(f'  test data:  X_test.shape ={ShapeToString(X_test)}, y_test.shape ={ShapeToString(y_test)}')
#     print()
    
#     return X_train, X_test, y_train, y_test

def TryKerasImport(verbose=True):
    
    kerasok = True
    try:
        import keras as keras_try
    except:
        kerasok = False

    tensorflowkerasok = True
    try:
        import tensorflow.keras as tensorflowkeras_try
    except:
        tensorflowkerasok = False
        
    ok = kerasok or tensorflowkerasok
    
    if not ok and verbose:
        if not kerasok:
            print("WARNING: importing 'keras' failed", file=sys.stderr)
        if not tensorflowkerasok:
            print("WARNING: importing 'tensorflow.keras' failed", file=sys.stderr)

    return ok
    
print(f"OK(function setup" + ("" if TryKerasImport() else ", hope MNIST loads works because it seems you miss the installation of Keras or Tensorflow!") + ")")

OK(function setup)


In [ ]:
from sklearn.utils import shuffle

# Træning:
print("x_test: ", X_test.shape)
print("y_test: ", y_test.shape)
print("x_train: ", X_train.shape)
print("y_train: ", y_train.shape)

# Shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# Så bygger vi CNN modellen:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binær classification (is / not-is)

model.compile(optimizer='adam',
              loss='binary_crossentropy', # also hvad pokker er det her
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
model.summary()



x_test:  (200, 512, 512, 3)
y_test:  (200,)
x_train:  (1800, 512, 512, 3)
y_train:  (1800,)
Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.5322 - loss: 2.1089 - val_accuracy: 0.6850 - val_loss: 0.6503
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 186s 3s/step - accuracy: 0.6695 - loss: 0.6343 - val_accuracy: 0.6600 - val_loss: 0.6350
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 193s 3s/step - accuracy: 0.6936 - loss: 0.6009 - val_accuracy: 0.6800 - val_loss: 0.6031
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 194s 3s/step - accuracy: 0.7589 - loss: 0.4954 - val_accuracy: 0.7600 - val_loss: 0.4887
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 188s 3s/step - accuracy: 0.8520 - loss: 0.3529 - val_accuracy: 0.6750 - val_loss: 0.7139
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 194s 3s/step - accuracy: 0.9463 - loss: 0.1471 - val_accuracy: 0.6850 - val_loss: 0.7455
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accuracy: 0.9813 - loss: 0.0689 - val_accuracy: 0.7450 - val_loss: 0.9615
Epoch 8/10
57/57 ━━

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 124, 124, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 984064)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │    62,980,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 189,109,637 (721.40 MB)

 Trainable params: 63,036,545 (240.47 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 126,073,092 (480.93 MB)

In [12]:
!pip install scikeras
!pip install --upgrade keras
!pip install --upgrade tensorflow

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.9.0
    Uninstalling keras-3.9.0:
      Successfully uninstalled keras-3.9.0

  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\morte\\anaconda3\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Consider using the `--user` option or check the permissions.




  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 4.2/376.0 MB 27.9 MB/s eta 0:00:14
    --------------------------------------- 8.9/376.0 MB 25.1 MB/s eta 0:00:15
   -- ------------------------------------- 22.0/376.0 MB 39.8 MB/s eta 0:00:09
   --- ------------------------------------ 35.4/376.0 MB 45.9 MB/s eta 0:00:08
   ----- ---------------------------------- 47.2/376.0 MB 47.7 MB/s eta 0:00:07
   ------ --------------------------------- 60.0/376.0 MB 49.7 MB/s eta 0:00:07
   ------- -------------------------------- 72.4/376.0 MB 51.3 MB/s eta 0:00:06
   -------- ------------------------------- 84.1/376.0 MB 51.6 MB/s eta 0:00:06
   ---------- ----------------------------- 97.8/376.0 MB 52.9 MB/s eta 0:00:06
   ----------- --------------------------- 110.1/376.0 MB 53.7 MB/s eta 0:00:05
   ------------ -------------------------- 123.7/376.0 M

In [ ]:
# Parameter udvalg (ved hjælp af random serarch)
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from time import time
from scikeras.wrappers import KerasClassifier  # modern wrapper for keras + sklearn

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD

# IMG_SIZE er 512
def build_model(optimizer='adam', activation='relu'):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation=activation, input_shape=(IMG_SIZE, IMG_SIZE, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation=activation))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation=activation))
    model.add(Flatten())
    model.add(Dense(64, activation=activation))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


# wrapper med keras
keras_clf = KerasClassifier(
    model=build_model, 
    optimizer="adam", 
    activation="relu", 
    verbose=0
)

# hyperparametre
param_grid = {
    "batch_size": [16, 32, 64],
    "epochs": [5, 10],
    "optimizer": ['adam', 'sgd'],
    "activation": ['relu' ]# , 'tanh']
}

# random search
random_tuned = RandomizedSearchCV(
    estimator=keras_clf,
    param_distributions=param_grid,
    n_iter=5,
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42
)

start = time()
random_tuned.fit(X_train, y_train)
t = time() - start

# find den bedste model
y_pred = random_tuned.predict(X_test)
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))
print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))
print(f"\nBest Parameters: {random_tuned.best_params_}")
print(f"Search took {t:.2f} seconds.")




# # optional parametre hvis den skal prøve nogle flere:
# 'dropout_rate': [0.0, 0.2, 0.5]


Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\Users\morte\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
